In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os; os.chdir('C:\\work\\dev\\python\\progs\\scraper\\ufc\\')

In [3]:
import pandas as pd

# Все тесты

In [75]:
import unittest

dn = 'tests'
loader = unittest.TestLoader()
suite = loader.discover(start_dir=dn, pattern=r'test_*', top_level_dir='.')

runner = unittest.TextTestRunner()
runner.run(suite)

..s.....C:\work\dev\python\progs\scraper\ufc\src\prep_funcs.py:94: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fights_in_fn)
........
----------------------------------------------------------------------
Ran 16 tests in 24.894s

OK (skipped=1)


<unittest.runner.TextTestResult run=16 errors=0 failures=0>

# Один тест

In [143]:
%run tests/test_fighters_feat_gen

C:\tools\Anaconda3\envs\develop_env\lib\site-packages\pandas\core\flags.py:51: ResourceWarning: unclosed file <_io.FileIO name='params.yaml' mode='rb' closefd=True>
  self._obj = weakref.ref(obj)
.
----------------------------------------------------------------------
Ran 1 test in 0.067s

OK


# Manual tests

In [4]:
from ruamel.yaml import YAML
import pandas as pd
import numpy as np
import joblib

In [5]:
conf = YAML().load(open('params.yaml'))

Если ranks_name = NULL что подтянется

## inference/train same results


In [17]:
feat_df = pd.read_csv(conf['split_data']['feat_full_fn'])
# pred_feat_df = feat_df[(feat_df.fighter.str.contains('desanya'))&(feat_df.opponent.str.contains('mavov'))].sort_values(by='event_day')

fighter1="Reinier de Ridder"
fighter2="Robert Whittaker"
pred_feat_df = feat_df.query('fighter==@fighter1 and opponent==@fighter2')
pred_feat_df

,event,fighter,opponent,event_day,target,split,left_corner_custom_feat,height_diff_custom_feat,reach_diff_custom_feat,dob_diff_custom_feat,...,fighter_stance_custom_feat_Orthodox,fighter_stance_custom_feat_Sideways,fighter_stance_custom_feat_Southpaw,fighter_stance_custom_feat_Switch,fighter_stance_custom_feat_nan,opponent_stance_custom_feat_Orthodox,opponent_stance_custom_feat_Sideways,opponent_stance_custom_feat_Southpaw,opponent_stance_custom_feat_Switch,opponent_stance_custom_feat_nan
8085,UFC Fight Night: Whittaker vs. De Ridder,Reinier de Ridder,Robert Whittaker,2025-07-26,1,ts,0,4.0,5.0,-3.466667,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [18]:
model = joblib.load(conf['train_eval']['model_fn'])
feats = joblib.load(conf['feat_shortlist_sel']['sign_feat_short_fn'])

pred_feat_df['score1'] = model.predict_proba(pred_feat_df[feats])[:,1]
pred_feat_df

C:\Users\admin\AppData\Local\Temp\ipykernel_13460\3203226877.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_feat_df['score1'] = model.predict_proba(pred_feat_df[feats])[:,1]


,event,fighter,opponent,event_day,target,split,left_corner_custom_feat,height_diff_custom_feat,reach_diff_custom_feat,dob_diff_custom_feat,...,fighter_stance_custom_feat_Sideways,fighter_stance_custom_feat_Southpaw,fighter_stance_custom_feat_Switch,fighter_stance_custom_feat_nan,opponent_stance_custom_feat_Orthodox,opponent_stance_custom_feat_Sideways,opponent_stance_custom_feat_Southpaw,opponent_stance_custom_feat_Switch,opponent_stance_custom_feat_nan,score1
8085,UFC Fight Night: Whittaker vs. De Ridder,Reinier de Ridder,Robert Whittaker,2025-07-26,1,ts,0,4.0,5.0,-3.466667,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.512829


In [8]:
from src.inference import get_features_event

pred_df = get_features_event('UFC Fight Night: Уиттакер - Де Риддер', 
                                    fights_fn=conf['inference']['fights_fn'], 
                                    fighters_fn=conf['inference']['fighters_fn'], 
                                    coef_fn=conf['inference']['coef_fn'], 
                                    vocab_fn=conf['inference']['vocab_fn'], 
                                    ufc_rankings_fn=conf['inference']['rankings_fn'], 
                                    rankings_big_fn=conf['inference']['rankings_big_fn'], conf=conf)

C:\work\dev\python\progs\scraper\ufc\src\prep_funcs.py:94: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fights_in_fn)


In [19]:
inf_df = pred_df.query('fighter==@fighter1 and opponent==@fighter2')
if inf_df.shape[0]==0:
    inv_f = True
    inf_df = pred_df.query('fighter==@fighter2 and opponent==@fighter1')
else:
    inv_f = False

In [20]:
inf_df

,event,fighter,opponent,event_day,coef1,coef2,left_corner_custom_feat,height_diff_custom_feat,reach_diff_custom_feat,dob_diff_custom_feat,...,fighter_stance_custom_feat_Sideways,fighter_stance_custom_feat_Southpaw,fighter_stance_custom_feat_Switch,fighter_stance_custom_feat_nan,opponent_stance_custom_feat_Orthodox,opponent_stance_custom_feat_Sideways,opponent_stance_custom_feat_Southpaw,opponent_stance_custom_feat_Switch,opponent_stance_custom_feat_nan,score1
6,UFC Fight Night: Уиттакер - Де Риддер,Robert Whittaker,Reinier de Ridder,2025-07-26,1.66,2.19,1.0,-4,-5.0,3.466667,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.521189


In [23]:
# тут есть и другие бои, плюс порядок бойцов другой, наоборот
inf_df['score1'] = model.predict_proba(inf_df[feats])[:,1]

C:\Users\admin\AppData\Local\Temp\ipykernel_13460\4143722866.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inf_df['score1'] = model.predict_proba(inf_df[feats])[:,1]


In [26]:
inf_df[feats+['score1']].iloc[-1]

dob_diff_custom_feat    3.466667
score1                  0.521189
Name: 6, dtype: float64

In [27]:
pred_feat_df[feats+['score1']].iloc[0]

dob_diff_custom_feat   -3.466667
score1                  0.512829
Name: 8085, dtype: float64

Построил логистическую регрессию для предсказания победы бойца в ufc (ничью не рассматриваю). Однако, если передавать бойцов в обратном порядке, 

## корректность подсчета ставок

In [19]:
fake_df = pd.DataFrame({'score1':[0.2, 0.5, 0.1, 0.8, 0.4, 0.1], 'target':[1,0,0,0,1,1], 
              'event_day':[pd.Period(it, freq='D') for it in ['2025-11-01', '2025-11-10', '2025-11-01', '2025-11-01', '2025-11-10', '2025-11-10']], 
              'coef1':[1.2,3,3.3,2.5,2.6, 2.25], 'coef2':[2.1, 1.21, 1.1,1.51,1.7,1.4]})

In [123]:
from src.inference import place_bet, calc_profit, calc_time_profit

alpha = 0.6
bet_df = place_bet(placebet_df=fake_df, strategy_selection=None, alpha=alpha)


# взяли недооцененного фаворита
row1 = bet_df.iloc[0]

diff = row1['proba2'] - row1['score2']
assert diff.round(2) == -0.44
assert row1['betwin'] == (diff<0)

# с таким же betwin, суммируем diff
sum_diffs = bet_df.loc[bet_df.betwin==row1['betwin'], 'diff'].abs().sum()
assert (-diff/sum_diffs*alpha).round(2) == row1['bet'].round(2)

# взяли недооцененного аутсайдера
row2 = bet_df.iloc[1]

diff = row2['proba1'] - row2['score1']
assert diff == row2['diff']

# с таким же betwin, суммируем diff
sum_diffs = bet_df.loc[bet_df.betwin==row2['betwin'], 'diff'].abs().sum()
assert ((-diff/sum_diffs)*(1-alpha)).round(2) == row2['bet'].round(2)

### во времени

In [ ]:
income_time_df, res_l = calc_time_profit(fake_df, strategy_selection=None, alpha=alpha)

t_df = fake_df[fake_df.event_day=='2025-11-01']
t_bet_df = place_bet(placebet_df=t_df, strategy_selection=None, alpha=alpha)
row = t_bet_df.iloc[1]

assert (row['diff']/t_bet_df['diff'].sum()).round(2) == 0.15
assert (t_bet_df['bet'].to_numpy() * np.array([2.1, 1.1, 2.5]))[1] == income_time_df['income'].iloc[0]



t_df = fake_df[fake_df.event_day=='2025-11-10']
t_bet_df = place_bet(placebet_df=t_df, strategy_selection=None, alpha=alpha)
assert income_time_df['income'].iloc[1] == t_bet_df['bet'].iloc[1]*t_bet_df['coef1'].iloc[1]


## inference

In [18]:
ev_nm = 'UFC Fight Night: Уиттакер - Де Риддер'

coef_df = pd.read_excel(conf['inference']['coef_fn'])
coef_df = coef_df.assign(event_day=lambda x:pd.to_datetime(x['date'], format='%d.%m.%Y'))
# pred_df = coef_df[coef_df['name']==ev_nm]

# в словаре надо, чтобы бойцы были
pred_df = coef_df[coef_df.fighter1.str.contains('Адеса') & coef_df.fighter2.str.contains('Имав')]
pred_df = pred_df.assign(event_day='2025-02-01')

In [19]:
from src.inference import get_features_event

feat_pred_df = get_features_event(ev_nm=None, 
                            fights_fn=conf['inference']['fights_fn'], 
                            fighters_fn=conf['inference']['fighters_fn'], 
                            coef_fn=conf['inference']['coef_fn'], 
                            vocab_fn=conf['inference']['vocab_fn'], 
                            ufc_rankings_fn=conf['inference']['rankings_fn'], 
                            rankings_big_fn=conf['inference']['rankings_big_fn'], conf=conf, pred_df=pred_df)


feat_pred_df.iloc[:, :20]

C:\work\dev\python\progs\scraper\ufc\src\prep_funcs.py:94: DtypeWarning: Columns (205,206,207,208) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(fights_in_fn)


,fighter,opponent,event_day,left_corner_custom_feat,height_custom_feat,reach_custom_feat,dob_diff_custom_feat,fighter_stance_custom_feat,opponent_stance_custom_feat,fighter_dob_custom_feat,fighter_days_nofight_custom_feat,opponent_days_nofight_custom_feat,ctrl_dam_exp_max_min_d_feat,ctrl_dam_exp_sum_feat,ctrl_dam_rol_max_min_d_feat,ctrl_dam_rol_sum_feat,ctrl_exp_max_min_d_feat,ctrl_exp_sum_feat,ctrl_rol_max_min_d_feat,ctrl_rol_sum_feat
0,Israel Adesanya,Nassourdine Imavov,2025-02-01,NaN,1,5.0,-68.266667,Switch,Orthodox,432.6,168.0,126.0,0.069155,0.650684,-0.203448,-0.589063,-0.092612,-1.554694,-0.307083,-0.597247


In [8]:
feat_df = pd.read_csv(conf['fights_feat_gen']['feat_fn'])


feat_row = feat_df.query('fighter=="Reinier de Ridder" and opponent=="Robert Whittaker"')


pred_row = feat_pred_df.query('fighter=="Robert Whittaker" and opponent=="Reinier de Ridder"')

assert set([it for it in feat_row.columns if not it in pred_row.columns])==set(['event', 'target'])

assert len([it for it in pred_row.columns if not it in feat_row.columns])==0

assert np.abs(feat_row['dob_diff_custom_feat'].iloc[0].round(2)) == np.abs(pred_row['dob_diff_custom_feat'].iloc[0].round(2))

assert np.abs(feat_row['dob_diff_custom_feat'].iloc[0].round(2)) == np.abs(pred_row['dob_diff_custom_feat'].iloc[0].round(2))

assert np.abs(feat_row['kd_rol_sum_feat'].iloc[0].round(2)) == np.abs(pred_row['kd_rol_sum_feat'].iloc[0].round(2))


In [9]:
feat_row.iloc[:, :20]

,event,fighter,opponent,event_day,target,left_corner_custom_feat,height_custom_feat,reach_custom_feat,dob_diff_custom_feat,fighter_stance_custom_feat,opponent_stance_custom_feat,fighter_dob_custom_feat,fighter_days_nofight_custom_feat,opponent_days_nofight_custom_feat,ctrl_dam_exp_max_min_d_feat,ctrl_dam_exp_sum_feat,ctrl_dam_rol_max_min_d_feat,ctrl_dam_rol_sum_feat,ctrl_exp_max_min_d_feat,ctrl_exp_sum_feat
3421,UFC Fight Night: Whittaker vs. De Ridder,Reinier de Ridder,Robert Whittaker,2025-07-26,1,0,4.0,5.0,-3.466667,Southpaw,Orthodox,424.7,84.0,273.0,-0.585911,-1.906904,-0.585911,-0.340173,0.360245,-0.149679


In [10]:
pred_row.isna().sum().loc[lambda x: x>0], feat_row.isna().sum().loc[lambda x: x>0]

(Series([], dtype: int64), Series([], dtype: int64))

## fights_feat_gen

In [12]:
df = pd.read_csv(conf['preprocess']['source_fighters_fn'])


In [7]:
feat_df = pd.read_csv(conf['fights_feat_gen']['feat_fn'])
stat_df = pd.read_csv(conf['stat_feat_gen']['feat_fn'])

In [31]:
fighter1 = 'Ilia Topuria'
fighter2 = 'Josh Emmett'
feat_nm = 'ctrl_dam_stat_exp_max_min_d_sub'
feat_df.loc[(feat_df.fighter==fighter1) & (feat_df.opponent==fighter2), feat_nm].iloc[0]

-0.0266666666666665

In [32]:
# статистика относительно fighter1
stat_df.loc[(stat_df.Fighter==fighter1) & (stat_df.Opponent==fighter2), feat_nm.replace('_sub', '')].iloc[0]

0.19

In [33]:
stat_df.loc[(stat_df.Fighter==fighter2) & (stat_df.Opponent==fighter1), feat_nm.replace('_sub', '')].iloc[0]

0.2166666666666666

## stat_feat_gen

In [ ]:
stat_df = pd.read_csv(conf['stat_feat_gen']['feat_fn'])

In [11]:
df = pd.read_csv(conf['filter']['filt_fn'])
df.sort_values(by=['Fighter', 'Date'], inplace=True)

In [8]:
N = 6
stat_df.query('Fighter=="Conor McGregor"')['ctrl_stat_exp_max_min_d'].iloc[N].round(2)==0.57

True

In [12]:
(df.query('Fighter=="Conor McGregor"')['ctrl_stat'][:N].max() - df.query('Fighter=="Conor McGregor"')['ctrl_stat'][:N].min()).round(2)

0.57

___

In [13]:
N = 13
stat_df.query('Fighter=="Conor McGregor"')['lose_stat_rol_sum'].iloc[N]==2

True

In [14]:

df.query('Fighter=="Conor McGregor"')['lose_stat'].iloc[N-conf['stat_feat_gen']['rol_window_size']:N].sum()

2

___

In [15]:
stat_df.query('Fighter=="Conor McGregor"')['kd_stat_rol_sum'].iloc[3].round(2)==1.14

True

In [16]:
df.query('Fighter=="Conor McGregor"')['kd_stat'].iloc[:3].sum().round(2)

1.14

## corner stat

In [36]:
coef_df = pd.read_csv(conf['preprocess']['prep_coef_fn'])
coef_df.query('date=="21.06.2025"')

,fighter1,fighter2,coef1,coef2,date,name,load_date,source,fighters,event_day,susp,corner
1206,Джамал Хилл,Халил Рантри,2.20,1.60,21.06.2025,NaN,NaN,NaN,Джамал Хилл Халил Рантри,2025-06-21,0,left
1216,Обан Эллиотт,Ко Сокхьон,4.90,1.17,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Ко Сокхьон Обан Эллиотт,2025-06-21,1,left
1217,Хамди Абдельвахаб,Мохаммед Усман,1.62,2.30,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Мохаммед Усман Хамди Абдельвахаб,2025-06-21,1,left
1218,Тагир Уланбеков,Азат Максум,1.20,4.60,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Азат Максум Тагир Уланбеков,2025-06-21,1,left
1219,Ирина Алексеева,"Сигула, Клаудия",1.37,3.15,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,"Ирина Алексеева Сигула, Клаудия",2025-06-21,1,left
1220,Дарья Железнякова,Мелисса Маллинз,3.05,1.38,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Дарья Железнякова Мелисса Маллинз,2025-06-21,1,left
1221,Юн Йон Пак,Исмаил Наурдиев,2.48,1.55,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Исмаил Наурдиев Юн Йон Пак,2025-06-21,1,left
1222,Обан Эллиотт,"Ко, Сок Хюн",4.60,1.20,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,"Ко, Сок Хюн Обан Эллиотт",2025-06-21,1,left
1223,Мухаммад Наимов,Богдан Град,1.37,3.15,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Богдан Град Мухаммад Наимов,2025-06-21,1,left
1224,Назим Садыков,Николас Мотта,1.23,4.20,21.06.2025,UFC Fight Night: Хилл - Раунтри,NaN,NaN,Назим Садыков Николас Мотта,2025-06-21,1,left


In [51]:
fights_df = pd.read_csv(conf['fights_feat_gen']['feat_fn'])

In [52]:
fights_df.loc[fights_df.fighter=='Ilia Topuria']

,fighter,opponent,event_date,target,left_corner_stat,kd_stat_rol_sum_sub,kd_dam_stat_rol_sum_sub,sub_att_stat_rol_sum_sub,sub_att_dam_stat_rol_sum_sub,rev_stat_rol_sum_sub,...,ctrl_stat_exp_max_min_d_sub,ctrl_dam_stat_exp_max_min_d_sub,win_stat_exp_max_min_d_sub,lose_stat_exp_max_min_d_sub,wrest_w_stat_exp_max_min_d_sub,wrest_l_stat_exp_max_min_d_sub,ko_w_stat_exp_max_min_d_sub,ko_l_stat_exp_max_min_d_sub,dec_w_stat_exp_max_min_d_sub,dec_l_stat_exp_max_min_d_sub
2253,Ilia Topuria,Jai Herbert,2022-03-19,1,0,-0.066491,-0.086957,0.333333,-0.211776,0.000000,...,0.240650,-0.602683,-1.0,-1.0,0.0,-1.0,0.0,-1.0,1.0,0.0
2254,Ilia Topuria,Josh Emmett,2023-06-24,1,0,0.219270,0.163488,0.122449,-0.107335,-0.066667,...,0.192776,-0.026667,-1.0,-1.0,1.0,-1.0,0.0,-1.0,0.0,-1.0
2255,Ilia Topuria,Max Holloway,2024-10-26,1,1,0.046993,0.000000,0.026141,0.000000,0.000000,...,0.257030,-0.248424,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,-1.0


In [58]:
fights_df.loc[fights_df.fighter=='Alexander Volkanovski']

,fighter,opponent,event_date,target,left_corner_stat,kd_stat_rol_sum_sub,kd_dam_stat_rol_sum_sub,sub_att_stat_rol_sum_sub,sub_att_dam_stat_rol_sum_sub,rev_stat_rol_sum_sub,...,ctrl_stat_exp_max_min_d_sub,ctrl_dam_stat_exp_max_min_d_sub,win_stat_exp_max_min_d_sub,lose_stat_exp_max_min_d_sub,wrest_w_stat_exp_max_min_d_sub,wrest_l_stat_exp_max_min_d_sub,ko_w_stat_exp_max_min_d_sub,ko_l_stat_exp_max_min_d_sub,dec_w_stat_exp_max_min_d_sub,dec_l_stat_exp_max_min_d_sub
215,Alexander Volkanovski,Jeremy Kennedy,2018-02-10,1,1,0.066667,-0.066667,0.000000,0.00,0.140845,...,-0.103991,-0.044272,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
216,Alexander Volkanovski,Darren Elkins,2018-07-14,1,0,-0.008427,0.000000,-0.344174,0.00,-0.075094,...,-0.216225,-0.466234,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0
217,Alexander Volkanovski,Chad Mendes,2018-12-29,1,0,-0.215504,-0.505908,0.200000,0.00,0.000000,...,0.034657,0.169950,-1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0
218,Alexander Volkanovski,Jose Aldo,2019-05-11,1,0,0.009976,0.108303,0.200000,0.00,0.000000,...,0.412538,-0.030050,-1.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0
219,Alexander Volkanovski,Max Holloway,2019-12-14,1,0,0.241637,0.108303,0.160000,0.00,0.000000,...,0.551204,-0.213807,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0
220,Alexander Volkanovski,Max Holloway,2020-07-11,1,1,0.108303,0.108303,-0.040000,0.00,0.000000,...,0.551204,-0.207363,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0
221,Alexander Volkanovski,Brian Ortega,2021-09-25,1,1,-0.291268,0.000000,0.000000,0.00,0.000000,...,0.053932,-0.361723,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0
222,Alexander Volkanovski,Chan Sung Jung,2022-04-09,1,1,-0.303030,-0.080000,-0.120000,0.12,0.080000,...,0.332929,-0.434272,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,-1.0
223,Alexander Volkanovski,Max Holloway,2022-07-02,1,1,0.063492,0.000000,-0.040000,0.12,0.080000,...,0.508538,-0.206029,-1.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,-1.0
224,Alexander Volkanovski,Islam Makhachev,2023-02-11,0,0,-0.057476,0.000000,-0.948554,0.12,0.080000,...,-0.013303,-0.038717,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0


In [55]:
stat_df.loc[stat_df.Fighter=='Ilia Topuria', ['Event', 'event_date' , 'Fighter', 'Opponent'] + [it for it in stat_df.columns if 'left' in it]]

,Event,event_date,Fighter,Opponent,left_corner_stat,left_corner_stat_rol_sum,left_corner_stat_exp_sum,left_corner_stat_rol_max_min_d,left_corner_stat_exp_max_min_d
6339,UFC Fight Night: Moraes vs. Sandhagen,2020-10-10,Ilia Topuria,Youssef Zalal,0,NaN,NaN,NaN,NaN
6340,UFC Fight Night: Hermansson vs. Vettori,2020-12-05,Ilia Topuria,Damon Jackson,1,NaN,NaN,NaN,NaN
6341,UFC 264: Poirier vs. McGregor 3,2021-07-10,Ilia Topuria,Ryan Hall,0,NaN,NaN,NaN,NaN
6342,UFC Fight Night: Volkov vs. Aspinall,2022-03-19,Ilia Topuria,Jai Herbert,0,1.0,1.0,1.0,1.0
6343,UFC 282: Blachowicz vs. Ankalaev,2022-12-10,Ilia Topuria,Bryce Mitchell,0,1.0,1.0,1.0,1.0
6344,UFC Fight Night: Emmett vs. Topuria,2023-06-24,Ilia Topuria,Josh Emmett,0,0.0,1.0,0.0,1.0
6345,UFC 298: Volkanovski vs. Topuria,2024-02-17,Ilia Topuria,Alexander Volkanovski,0,0.0,1.0,0.0,1.0
6346,UFC 308: Topuria vs. Holloway,2024-10-26,Ilia Topuria,Max Holloway,1,0.0,1.0,0.0,1.0


## old feat tests

In [5]:
from ruamel.yaml import YAML
conf = YAML().load(open('params.yaml'))

In [8]:
feat_df = pd.read_csv(conf['stat_feat_gen']['feat_fn'])

In [9]:
import re
from src.funcs import get_rolling_stats

df = pd.read_csv(conf['filter']['filt_fn'])

df.sort_values(by=['Fighter', 'Date'], inplace=True)
stat_cols = [it for it in df.columns if re.search('_stat', it)]
    
op_d = {col:'median' for col in stat_cols if re.search('str',col)}
op_d.update({col:'mean' for col in stat_cols if not re.search('str',col)})

In [17]:
df_stat = df.query('Fighter.isin(["Khabib Nurmagomedov", "Ben Askren"])').groupby('Fighter').apply(lambda df: get_rolling_stats(df,op_d, conf['stat_feat_gen']['min_fights_num']))

In [ ]:
df_fighter.expanding(min_periods=min_fights_num).agg(op_d)

In [23]:
import numpy as np
np.median([2.8846151,1.666667,11.478261, 1.533333])

2.27564105

In [16]:
df.loc[df.Fighter=='Khabib Nurmagomedov', ['Event', 'sig_str_stat', 'Date', 'Opponent']]

,Event,sig_str_stat,Date,Opponent
10009,UFC on FX: Guillard vs Miller,2.884615,2012-01-20,Kamal Shalorus
10173,UFC 148: Silva vs Sonnen 2,1.666667,2012-07-07,Gleison Tibau
10335,UFC on FX: Belfort vs Bisping,11.478261,2013-01-19,Thiago Tavares
2289,UFC 160: Velasquez vs Silva 2,1.533333,2013-05-25,Abel Trujillo
10598,UFC 165: Jones vs Gustafsson,3.600000,2013-09-21,Pat Healy
10852,UFC on FOX: Werdum vs Browne,2.400000,2014-04-19,Rafael Dos Anjos
3624,UFC on FOX: Teixeira vs Evans,8.687259,2016-04-16,Darrell Horcher
3891,UFC 205: Alvarez vs McGregor,7.509987,2016-11-12,Michael Johnson
4450,UFC 219: Cyborg vs. Holm,5.933333,2017-12-30,Edson Barboza
4561,UFC 223: Khabib vs. Iaquinta,5.360000,2018-04-07,Al Iaquinta
